In [3]:
# Import Necessary data for exploring data 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
import calendar
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as offline
import plotly.graph_objs as go
offline.init_notebook_mode(connected = True)
pd.pandas.set_option('display.max_columns',None)
pd.options.display.float_format = '{:.2f}'.format

In [4]:
data = pd.read_csv(r'C:\Users\Propriétaire\Desktop\Final Project\cleaned_data.csv')
data.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.



,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,dcoilwtico,type_y,locale,locale_name,description,transferred,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0,Quito,Pichincha,D,13,93.14,Holiday,National,Ecuador,Primer dia del ano,False,0.00
1,1194,2013-01-01,42,CELEBRATION,0.00,0,Cuenca,Azuay,D,2,93.14,Holiday,National,Ecuador,Primer dia del ano,False,0.00
2,1193,2013-01-01,42,BREAD/BAKERY,0.00,0,Cuenca,Azuay,D,2,93.14,Holiday,National,Ecuador,Primer dia del ano,False,0.00
3,1192,2013-01-01,42,BOOKS,0.00,0,Cuenca,Azuay,D,2,93.14,Holiday,National,Ecuador,Primer dia del ano,False,0.00
4,1191,2013-01-01,42,BEVERAGES,0.00,0,Cuenca,Azuay,D,2,93.14,Holiday,National,Ecuador,Primer dia del ano,False,0.00


In [5]:
## changing data type from object to date 
data['date'] = data['date'].astype('datetime64[ns]')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 17 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            int64         
 1   date          datetime64[ns]
 2   store_nbr     int64         
 3   family        object        
 4   sales         float64       
 5   onpromotion   int64         
 6   city          object        
 7   state         object        
 8   type_x        object        
 9   cluster       int64         
 10  dcoilwtico    float64       
 11  type_y        object        
 12  locale        object        
 13  locale_name   object        
 14  description   object        
 15  transferred   object        
 16  transactions  float64       
dtypes: datetime64[ns](1), float64(3), int64(4), object(9)
memory usage: 396.1+ MB


### Sales Analysis 

In [7]:
## Creating new dataframe with groupby to have special to analys our data
data_sa = data.groupby('store_nbr').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=True)
data_sa_type_store = data.groupby('type_x').agg({"sales" : "mean"}).reset_index()
data_fa_sa = data.groupby('family').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=True)[:10]
data_cl_sa = data.groupby('cluster').agg({"sales" : "mean"}).reset_index() 
data_tm = data.groupby('date').agg({"sales" : "sum"}).reset_index()

In [5]:
fig = px.line(data_tm, x="date", y="sales", title='Sales Evolution by Time')
fig.show()

In [8]:
## Create new column in new dataset to have new columns of year, month, weeek.. to make analysis 
data_tm['year'] = data_tm['date'].dt.year
data_tm['month'] = data_tm['date'].dt.month
data_tm['week'] = data_tm['date'].dt.isocalendar().week
data_tm['quarter'] = data_tm['date'].dt.quarter
data_tm['day_of_week'] = data_tm['date'].dt.day_name()

In [9]:
# data 
data_2013 = data_tm[data_tm['year']==2013][['month','sales']]
data_2013 = data_2013.groupby('month').agg({"sales" : "mean"}).reset_index().rename(columns={'sales':'s13'})
data_2014 = data_tm[data_tm['year']==2014][['month','sales']]
data_2014 = data_2014.groupby('month').agg({"sales" : "mean"}).reset_index().rename(columns={'sales':'s14'})
data_2015 = data_tm[data_tm['year']==2015][['month','sales']]
data_2015 = data_2015.groupby('month').agg({"sales" : "mean"}).reset_index().rename(columns={'sales':'s15'})
data_2016 = data_tm[data_tm['year']==2016][['month','sales']]
data_2016 = data_2016.groupby('month').agg({"sales" : "mean"}).reset_index().rename(columns={'sales':'s16'})
data_2017 = data_tm[data_tm['year']==2017][['month','sales']]
data_2017 = data_2017.groupby('month').agg({"sales" : "mean"}).reset_index()
data_2017_no = pd.DataFrame({'month': [9,10,11,12], 'sales':[0,0,0,0]})
data_2017 = data_2017.append(data_2017_no).rename(columns={'sales':'s17'})
data_year = data_2013.merge(data_2014,on='month').merge(data_2015,on='month').merge(data_2016,on='month').merge(data_2017,on='month')

In [10]:
# top levels
top_labels = ['2013', '2014', '2015', '2016', '2017']

colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
          'rgba(122, 120, 168, 0.8)', 'rgba(164, 163, 204, 0.85)',
          'rgba(190, 192, 213, 1)']

# X axis value 
data_year = data_year[['s13','s14','s15','s16','s17']].replace(np.nan,0)
x_data = data_year.values

# y axis value (Month)
data_2013['month'] =['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
y_data = data_2013['month'].tolist()

fig = go.Figure()
for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        fig.add_trace(go.Bar(
            x=[xd[i]], y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(color='rgb(248, 248, 249)', width=1)
            )
        ))

fig.update_layout(title='Avg Sales by month for each Year',
    xaxis=dict(showgrid=False, 
               zeroline=False, domain=[0.15, 1]),
    yaxis=dict(showgrid=False, showline=False,
               showticklabels=False, zeroline=False),
    barmode='stack', 
    template="plotly_white",
    margin=dict(l=0, r=50, t=100, b=10),
    showlegend=False, 
)

annotations = []
for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.1,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                          showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]
fig.update_layout(
    annotations=annotations)
fig.show()

In [13]:
# chart color
data_fa_sa['color'] = '#496595'
data_fa_sa['color'][:-2] = '#c6ccd8'
data_cl_sa['color'] = '#c6ccd8'

fig = make_subplots(rows=2, cols=2, 
                    specs=[[{"type": "bar"}, {"type": "pie"}],
                           [{"colspan": 2}, None]],
                    column_widths=[0.7, 0.3], vertical_spacing=0, horizontal_spacing=0.02,
                    subplot_titles=("Top 10 Highest Product Sales", "Highest Sales by store clusters", "Clusters Vs Sales"))


fig.add_trace(go.Bar(x=data_fa_sa['sales'], y=data_fa_sa['family'], marker=dict(color= data_fa_sa['color']),
                     name='Family', orientation='h'), 
                     row=1, col=1)


fig.add_trace(go.Pie(values=data_sa_type_store['sales'], labels=data_sa_type_store['type_x'], name='Store type',
                     marker=dict(colors=['#334668','#496595','#6D83AA','#91A2BF','#C8D0DF']), hole=0.7,
                     hoverinfo='label+percent+value', textinfo='label'), 
                    row=1, col=2)



In [10]:
oil = pd.read_csv(r'C:\Users\Propriétaire\Desktop\Final Project\oil.csv')

In [11]:
p = oil.melt(id_vars=['date']+list(oil.keys()[5:]), var_name='Legend')
px.line(p.sort_values(["Legend", "date"], ascending = [False, True]), x='date', y='value', color='Legend',title = "Daily Oil Price" )

In [12]:
transactions = pd.read_csv(r'C:\Users\Propriétaire\Desktop\Final Project\transactions.csv')

In [13]:
transactions['date'] = transactions['date'].astype('datetime64[ns]')

In [14]:
df = transactions.copy()
df["year"] = df.date.dt.year
df["dayofweek"] = df.date.dt.dayofweek+1
df = df.groupby(["year", "dayofweek"]).transactions.mean().reset_index()
px.line(df, x="dayofweek", y="transactions" , color = "year", title = "Transactions")

In [15]:
data_st_event = data.groupby(['city','type_y']).agg({"sales" : "mean"}).reset_index()
data_st_event['sales'] = round(data_st_event['sales'], 2)

# chart
fig = px.scatter(data_st_event, x='city', color='sales', y='type_y', size='sales',
                 color_discrete_sequence=px.colors.qualitative.D3,
                 title="Average Sales: City Store  Vs Events")
# styling
fig.update_yaxes(ticksuffix='  ')
fig.update_layout(height=400, xaxis_title='', yaxis_title='',
                  margin=dict(b=0),
                  plot_bgcolor='#fafafa', paper_bgcolor='#fafafa',
                  title_font=dict(size=29, color='#444', family="Lato, sans-serif"),
                  font=dict(color='#555'), 
                  hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif"))
fig.show()